In [1]:
import pandas as pd 
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [2]:
password = getpass.getpass()

········


In [ ]:
#1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X).
#2. Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our **TARGET** (y) variable.
#3. Read the data into a Pandas dataframe.  At this point you should have 1000 rows.  Number of columns depends on the number of features you chose.

In [3]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT 
    f.film_id,
    f.title,
    f.length,
    f.rating,
    f.rental_rate,
    f.rental_duration,
    MAX(CASE WHEN MONTH(r.rental_date) = 5 AND YEAR(r.rental_date) = 2005 THEN True ELSE False END) AS rented_in_may,
    COUNT(r.rental_id) AS rental_count,
    AVG(f.rental_duration) AS avg_rental_duration,
    AVG(f.rental_rate) AS avg_rental_rate,
    AVG(r.rental_date) AS avg_rental_date
FROM film f
LEFT JOIN inventory i ON i.film_id = f.film_id
LEFT JOIN rental r ON r.inventory_id = i.inventory_id
GROUP BY f.film_id;
'''
df = pd.read_sql(query, connection_string)

In [4]:
df

,film_id,title,length,rating,rental_rate,rental_duration,rented_in_may,rental_count,avg_rental_duration,avg_rental_rate,avg_rental_date
0,1,ACADEMY DINOSAUR,86,PG,0.99,6,1,23,6.0,0.99,2.005073e+13
1,2,ACE GOLDFINGER,48,G,4.99,3,0,7,3.0,4.99,2.005213e+13
2,3,ADAPTATION HOLES,50,NC-17,2.99,7,1,12,7.0,2.99,2.005074e+13
3,4,AFFAIR PREJUDICE,117,G,2.99,5,1,23,5.0,2.99,2.005113e+13
4,5,AFRICAN EGG,130,G,2.99,6,1,12,6.0,2.99,2.005150e+13
...,...,...,...,...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,183,G,0.99,6,0,7,6.0,0.99,2.005074e+13
996,997,YOUTH KICK,179,NC-17,0.99,4,0,6,4.0,0.99,2.005075e+13
997,998,ZHIVAGO CORE,105,NC-17,0.99,6,1,9,6.0,0.99,2.005175e+13
998,999,ZOOLANDER FICTION,101,R,2.99,5,1,17,5.0,2.99,2.005073e+13


In [5]:
df.isna().sum()

film_id                 0
title                   0
length                  0
rating                  0
rental_rate             0
rental_duration         0
rented_in_may           0
rental_count            0
avg_rental_duration     0
avg_rental_rate         0
avg_rental_date        42
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.isna().sum()

film_id                0
title                  0
length                 0
rating                 0
rental_rate            0
rental_duration        0
rented_in_may          0
rental_count           0
avg_rental_duration    0
avg_rental_rate        0
avg_rental_date        0
dtype: int64

In [8]:
df.reset_index(drop=True, inplace=True)

In [ ]:
#4. Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.
#5. Create a logistic regression model to predict 'rented_in_may' from the cleaned data.
#6. Evaluate the results.

In [11]:
categorical_columns = ['title', 'rating']  
numerical_columns = ['length', 'rental_rate', 'rental_duration', 'rental_count', 'avg_rental_duration', 'avg_rental_rate', 'avg_rental_date']

print("First Few Rows:", df.head())

X = df.drop('rented_in_may', axis=1)
y = df['rented_in_may']

if 'rating' in X.columns:
    encoder = OneHotEncoder(drop='first', sparse=False)
    X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_columns]), columns=encoder.get_feature_names_out(categorical_columns))
    X_encoded = pd.concat([X.drop(categorical_columns, axis=1), X_encoded], axis=1)

    numerical_columns_existing = [col for col in numerical_columns if col in X.columns]
    X_numerical = X[numerical_columns_existing]

    X_final = pd.concat([X_encoded, X_numerical], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy}')
    print('Classification Report:\n', classification_report(y_test, y_pred))
else:
    print("'rating' column not found in the DataFrame.")

First Few Rows:    film_id             title  length rating  rental_rate  rental_duration  \
0        1  ACADEMY DINOSAUR      86     PG         0.99                6   
1        2    ACE GOLDFINGER      48      G         4.99                3   
2        3  ADAPTATION HOLES      50  NC-17         2.99                7   
3        4  AFFAIR PREJUDICE     117      G         2.99                5   
4        5       AFRICAN EGG     130      G         2.99                6   

   rented_in_may  rental_count  avg_rental_duration  avg_rental_rate  \
0              1            23                  6.0             0.99   
1              0             7                  3.0             4.99   
2              1            12                  7.0             2.99   
3              1            23                  5.0             2.99   
4              1            12                  6.0             2.99   

   avg_rental_date  
0     2.005073e+13  
1     2.005213e+13  
2     2.005074e+13  
3   

C:\Users\franc\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\franc\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\franc\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\franc\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetr